In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!pip install nbstripout
!nbstripout /content/drive/MyDrive/2025/tcc-final/fine_tuning_Bertimbau_version2_24_07.ipynb

Could not strip '/content/drive/MyDrive/2025/tcc-final/fine_tuning_Bertimbau_version2_24_07.ipynb': file not found


In [23]:
!pip install transformers evaluate accelerate
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import evaluate
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback # Importa explicitamente para o callback
)

# **# 1. Carregar Dataset Processado**

Aqui além de carregar os dataset, também é feita a divisão dos dados em treino, validação e testes

In [24]:
# Carregar dataset já processado
file_path = '/content/drive/MyDrive/2025/tcc-final/denuncias_balanceadas.xlsx'
df = pd.read_excel(file_path)

print(f"Dataset carregado. Total de registros: {len(df)}")
print("Primeiras 5 linhas do dataset:")
print(df.head())
print("\nInformações do dataset:")
print(df.info())

"""# Dividir em treino (70%), validação (15%) e teste (15%)
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['classe'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['classe'], random_state=42)

print(f"Treino: {len(train_df)}, Validação: {len(val_df)}, Teste: {len(test_df)}")"""

Dataset carregado. Total de registros: 169
Primeiras 5 linhas do dataset:
                                               texto             classe
0  Segundo informações da [VÍTIMA], na sexta, a m...  invasao_domicilio
1  Policiais arrombaram a casa, quebraram sapatei...  invasao_domicilio
2  [VÍTIMA] relata que sua vizinha [VÍTIMA] teve ...  invasao_domicilio
3  [VÍTIMA] relata que abriram o portão de sua ca...  invasao_domicilio
4  [VÍTIMA] teve a casa invadida por policiais do...  invasao_domicilio

Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texto   169 non-null    object
 1   classe  169 non-null    object
dtypes: object(2)
memory usage: 2.8+ KB
None


'# Dividir em treino (70%), validação (15%) e teste (15%)\ntrain_df, temp_df = train_test_split(df, test_size=0.3, stratify=df[\'classe\'], random_state=42)\nval_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df[\'classe\'], random_state=42)\n\nprint(f"Treino: {len(train_df)}, Validação: {len(val_df)}, Teste: {len(test_df)}")'

# **2. Tokenização com BERTimbau**

Preparação os dados textuais (as denúncias) para serem compreendidos pelo modelo BERTimbau, convertendo o texto bruto em um formato numérico que o modelo pode processar.
*   AutoTokenizer é uma classe da biblioteca transformers que automaticamente seleciona a classe de tokenizador correta com base no nome do modelo.

*   Este tokenizador sabe como dividir o texto em "tokens" (palavras ou subpalavras), converter esses tokens em IDs numéricos (que o modelo entende), e adicionar tokens especiais que o BERT espera (como [CLS] e [SEP]).
* Além disso, a função tokenize_function receberá um "batch" (lote) de dados como entrada.



In [25]:
model_path = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_path)

def tokenize_function(batch):
    # 'padding='longest'' preenche cada sequência para o tamanho da sequência mais longa no lote atual,
    # em vez de um max_length fixo, o que é mais eficiente.
    # 'truncation=True' ainda corta sequências maiores que max_length.
    return tokenizer(batch["texto"], truncation=True, max_length=512, padding='longest', return_tensors='pt')

# Não tokenizamos o dataset completo aqui para K-Fold, faremos isso dentro do loop.
# As variáveis train_encodings, val_encodings, test_encodings serão criadas dentro do loop K-Fold.

# Mapeamento de Labels: Essencial para o modelo e métricas
id2label = {0: "invasao_domicilio", 1: "violencia_fisica"}
label2id = {v: k for k, v in id2label.items()}

# Custom Dataset (para trabalhar com tensores PyTorch diretamente)
class CustomDataset(Dataset): # Certifique-se que 'Dataset' foi importado corretamente
    def __init__(self, encodings, labels):
        self.encodings = encodings # Agora será um dicionário de tensores (input_ids, attention_mask)
        self.labels = labels       # Agora será uma lista de labels (ou Series com índice reiniciado)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        # val[idx] funciona diretamente em tensores ou listas de tensores.
        # labels.iloc[idx] é removido pois labels será uma lista/Series com índice reiniciado.
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long) # Acesso direto pelo índice
        return item

    def __len__(self):
        return len(self.labels)

# Data Collator para padding dinâmico durante o treinamento
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

"""# Função de tokenização
def tokenize_function(batch):
    return tokenizer(batch["texto"], truncation=True,max_length=512, padding=True,return_tensors='pt')

# Tokenizar os DataFrames diretamente (sem usar datasets.Dataset)
train_encodings = tokenize_function(train_df.to_dict('list'))
val_encodings = tokenize_function(val_df.to_dict('list'))
test_encodings = tokenize_function(test_df.to_dict('list'))"""

'# Função de tokenização\ndef tokenize_function(batch):\n    return tokenizer(batch["texto"], truncation=True,max_length=512, padding=True,return_tensors=\'pt\')\n\n# Tokenizar os DataFrames diretamente (sem usar datasets.Dataset)\ntrain_encodings = tokenize_function(train_df.to_dict(\'list\'))\nval_encodings = tokenize_function(val_df.to_dict(\'list\'))\ntest_encodings = tokenize_function(test_df.to_dict(\'list\'))'

# **3. Configurar o Modelo e congelamento**

Inicializar o modelo pré-treinado BERTimbau para uma tarefa de classificação de sequências e, crucialmente, configurar quais partes desse modelo serão treinadas (ajustadas) e quais permanecerão "congeladas".
* Mapeamento de Labels
* Carregamento e Configuração do Modelo de Classificação
* Congelamento de Camadas do BERT
 * for name, param in model.named_parameters():: Inicia um loop que itera sobre todos os parâmetros (pesos e vieses) do modelo. Para cada parâmetro, ele fornece seu name (nome da camada a que pertence) e o próprio param (o tensor do parâmetro).

 * if 'classifier' not in name and 'pooler' not in name:: Esta é a condição principal para o congelamento.

 * Ele verifica se o name do parâmetro não contém a string 'classifier' e não contém a string 'pooler'.

 * 'classifier': Refere-se aos parâmetros da cabeça de classificação que foi adicionada no topo do modelo BERT (a parte que faz a previsão das 2 classes).

 * 'pooler': Refere-se aos parâmetros da camada "pooler" do BERT. O pooler geralmente é usado para obter uma representação vetorial de tamanho fixo para toda a sequência de entrada. Para tarefas de classificação de sequência, o BERT frequentemente usa a saída do token [CLS] (o primeiro token) que passa pelo pooler.

In [26]:
"""id2label = {0: "invasao_domicilio", 1: "violencia_fisica"}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

# Congelar camadas do BERT (exceto pooler e classificador)
#for name, param in model.named_parameters():
#    if 'classifier' not in name and 'pooler' not in name:
#        param.requires_grad = False

# Congelar todas as camadas da base BERT
for param in model.bert.parameters():
    param.requires_grad = False

# Garantir que a cabeça de classificação seja treinável
# (Ela já é treinável por padrão se não for explicitamente congelada)
for param in model.classifier.parameters():
    param.requires_grad = True

# Opcional: Imprimir o número de parâmetros treináveis para verificar
print(f"Número de parâmetros treináveis: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")"""

'id2label = {0: "invasao_domicilio", 1: "violencia_fisica"}\nlabel2id = {v: k for k, v in id2label.items()}\n\nmodel = AutoModelForSequenceClassification.from_pretrained(\n    model_path,\n    num_labels=2,\n    id2label=id2label,\n    label2id=label2id\n)\n\n# Congelar camadas do BERT (exceto pooler e classificador)\n#for name, param in model.named_parameters():\n#    if \'classifier\' not in name and \'pooler\' not in name:\n#        param.requires_grad = False\n\n# Congelar todas as camadas da base BERT\nfor param in model.bert.parameters():\n    param.requires_grad = False\n\n# Garantir que a cabeça de classificação seja treinável\n# (Ela já é treinável por padrão se não for explicitamente congelada)\nfor param in model.classifier.parameters():\n    param.requires_grad = True\n\n# Opcional: Imprimir o número de parâmetros treináveis para verificar\nprint(f"Número de parâmetros treináveis: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")'

# **4. MÉTRICAS AVANÇADAS (Acurácia + AUC-ROC)**

In [27]:
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    try:
        predictions, labels = eval_pred

        # Converter logits em probabilidades usando softmax (para AUC-ROC)
        # np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True) é o softmax numérico
        probabilities = np.exp(predictions - np.max(predictions, axis=-1, keepdims=True)) / np.sum(np.exp(predictions - np.max(predictions, axis=-1, keepdims=True)), axis=-1, keepdims=True)
        positive_class_probs = probabilities[:, 1] # Probabilidade da classe positiva (ID 1)

        # Calcular métricas básicas
        preds = np.argmax(predictions, axis=1) # Classe prevista (ID da maior probabilidade)
        acc = accuracy.compute(predictions=preds, references=labels)["accuracy"]
        auc = auc_score.compute(prediction_scores=positive_class_probs, references=labels)["roc_auc"]

        # Calcular F1 para cada classe (average=None)
        f1_results = f1_metric.compute(
            predictions=preds,
            references=labels,
            average=None, # Calcula F1 para cada classe individualmente
            labels=[label2id["invasao_domicilio"], label2id["violencia_fisica"]] # IDs das classes
        )

        return {
            "accuracy": round(acc, 4),
            "auc": round(auc, 4),
            "f1_invasao": round(f1_results["f1"][0], 4),
            "f1_violencia": round(f1_results["f1"][1], 4)
        }
    except Exception as e:
        print(f"Erro no cálculo de métricas: {str(e)}")
        # Retorna 0s ou NaN em caso de erro para não quebrar o processo, mas indica problema
        return {"accuracy": 0.0, "auc": 0.0, "f1_invasao": 0.0, "f1_violencia": 0.0}

# **5. TREINAMENTO COM K-FOLD CROSS-VALIDATION E EARLY STOPPING**

In [28]:
# Parâmetros de Treinamento
training_args = TrainingArguments(
    output_dir="./bertimbau-denuncias-cv", # Diretório de saída para K-Fold

    # Configurações básicas
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    #learning_rate=2e-5,
    learning_rate=5e-5,
    num_train_epochs=30, # Aumentado para permitir que Early Stopping atue
    # Anteriormente: num_train_epochs=3, o que não dava espaço para Early Stopping

    # Estratégias
    eval_strategy="epoch",
    save_strategy="epoch",

    # Configurações de avaliação e salvamento
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss", # Monitorar a perda de validação
    greater_is_better=False, # Menor perda é melhor

    # Logging
    logging_dir="./logs-cv",
    logging_steps=10, # Diminuído para ver logs mais frequentemente (era 100)
    disable_tqdm=False,
    report_to="none",

    # Otimizações
    fp16=True, # Treinamento com precisão mista
    seed=42, # Semente para reprodutibilidade
    gradient_accumulation_steps=1,

    # Parâmetros específicos da v4.53.2
    remove_unused_columns=True,
    label_names=["labels"]
)

# Preparar dados para K-Fold
all_texts = df['texto'].tolist()
all_labels_mapped = df['classe'].map(label2id).tolist() # Labels já mapeadas para IDs numéricos

N_SPLITS = 5 # Número de folds. Para 150 registros, 5 folds é razoável.
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# Lista para armazenar as métricas de teste de cada fold
all_fold_test_metrics = []

print(f"\n🚀 Iniciando Treinamento com K-Fold Cross-Validation ({N_SPLITS} folds)...\n")

# Loop através dos folds
for fold, (train_index, test_index) in enumerate(skf.split(all_texts, all_labels_mapped)):
    print(f"\n==================== INICIANDO FOLD {fold+1}/{N_SPLITS} ====================")

    # Dividir os dados para o fold atual
    train_fold_texts = [all_texts[i] for i in train_index]
    train_fold_labels = [all_labels_mapped[i] for i in train_index]

    test_fold_texts = [all_texts[i] for i in test_index]
    test_fold_labels = [all_labels_mapped[i] for i in test_index]

    # Tokenizar os dados para o fold atual
    # Note: tokenize_function já retorna um dicionário de tensores PyTorch.
    train_fold_encodings = tokenize_function({"texto": train_fold_texts})
    test_fold_encodings = tokenize_function({"texto": test_fold_texts})

    # Criar CustomDatasets para o fold atual
    # Passamos os encodings (dicionário de tensores) e as labels (lista) diretamente
    # O CustomDataset agora espera isso e acessa por índice posicional.
    train_dataset = CustomDataset(
        train_fold_encodings, # Já é um dicionário de tensores
        train_fold_labels     # Já é uma lista
    )
    eval_dataset_for_trainer = CustomDataset(
        test_fold_encodings,
        test_fold_labels
    )

    # Re-instanciar o modelo para cada fold para garantir pesos iniciais limpos
    model = AutoModelForSequenceClassification.from_pretrained(
        model_path,
        num_labels=2,
        id2label=id2label,
        label2id=label2id
    )

    # Adicione ESTE BLOCO de CONGELAMENTO AQUI
    # Congelar camadas do BERT (exceto pooler e classificador)
    for name, param in model.named_parameters():
        if 'classifier' not in name and 'pooler' not in name:
            param.requires_grad = False

    # Opcional: Imprimir o número de parâmetros treináveis para verificar
    print(f"  Número de parâmetros treináveis no FOLD {fold+1}: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
    # ===============================================

    # Re-instanciar o Trainer para cada fold
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset_for_trainer, # Conjunto de teste/validação do fold
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)], # Aumentar paciência para 5 épocas
        compute_metrics=compute_metrics,
        data_collator=data_collator # Passado explicitamente
    )

    # Treinar o modelo para o fold atual
    try:
        train_result = trainer.train()
        print(f"📊 Métricas finais de treino para FOLD {fold+1}:")
        print(f"Loss: {train_result.metrics['train_loss']:.4f}")
        print(f"Tempo total: {train_result.metrics['train_runtime']:.2f}s")

        # Avaliar no conjunto de teste/validação do fold
        print(f"🧪 Avaliação no conjunto de teste/validação do FOLD {fold+1}...")
        fold_test_metrics = trainer.evaluate(eval_dataset_for_trainer)
        all_fold_test_metrics.append(fold_test_metrics)
        print(f"Resultados do FOLD {fold+1}: {fold_test_metrics}")

    except Exception as e:
        import traceback
        print(f"❌ Erro durante o treinamento do FOLD {fold+1}:")
        print(traceback.format_exc())
        # Em caso de erro, adiciona métricas vazias ou NaN para não parar a média final
        all_fold_test_metrics.append({
            'eval_loss': float('nan'),
            'eval_accuracy': 0.0,
            'eval_auc': 0.0,
            'eval_f1_invasao': 0.0,
            'eval_f1_violencia': 0.0,
            'eval_runtime': float('nan'), # Inclui outras métricas relevantes
            'eval_samples_per_second': 0.0,
            'eval_steps_per_second': 0.0
        })


🚀 Iniciando Treinamento com K-Fold Cross-Validation (5 folds)...


==================== INICIANDO FOLD 1/5 ====================


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Número de parâmetros treináveis no FOLD 1: 592130


Epoch,Training Loss,Validation Loss,Accuracy,Auc,F1 Invasao,F1 Violencia
1,0.697100,0.658275,0.676500,0.788900,0.521700,0.755600
2,0.653500,0.622393,0.735300,0.802800,0.709700,0.756800
3,0.604000,0.594123,0.735300,0.823500,0.709700,0.756800
4,0.556300,0.572582,0.735300,0.816600,0.709700,0.756800
5,0.544000,0.552361,0.735300,0.820100,0.709700,0.756800
6,0.500100,0.532729,0.764700,0.844300,0.750000,0.777800
7,0.495600,0.524172,0.705900,0.847800,0.666700,0.736800
8,0.509700,0.509753,0.735300,0.868500,0.689700,0.769200
9,0.466300,0.486396,0.764700,0.872000,0.750000,0.777800
10,0.460700,0.475388,0.764700,0.872000,0.750000,0.777800


📊 Métricas finais de treino para FOLD 1:
Loss: 0.4390
Tempo total: 201.36s
🧪 Avaliação no conjunto de teste/validação do FOLD 1...


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Número de parâmetros treináveis no FOLD 5: 592130


Epoch,Training Loss,Validation Loss,Accuracy,Auc,F1 Invasao,F1 Violencia
1,0.672600,0.652862,0.666700,0.739000,0.592600,0.717900
2,0.621200,0.630253,0.818200,0.750000,0.812500,0.823500
3,0.589900,0.609397,0.757600,0.783100,0.733300,0.777800
4,0.541100,0.591379,0.787900,0.786800,0.774200,0.800000
5,0.536800,0.576494,0.787900,0.801500,0.787900,0.787900
6,0.503200,0.560769,0.757600,0.808800,0.764700,0.750000
7,0.450400,0.546799,0.757600,0.808800,0.750000,0.764700
8,0.471300,0.533013,0.757600,0.816200,0.750000,0.764700
9,0.480100,0.520889,0.757600,0.827200,0.750000,0.764700
10,0.450700,0.516971,0.757600,0.838200,0.764700,0.750000


📊 Métricas finais de treino para FOLD 5:
Loss: 0.4141
Tempo total: 255.82s
🧪 Avaliação no conjunto de teste/validação do FOLD 5...


Resultados do FOLD 5: {'eval_loss': 0.430359810590744, 'eval_accuracy': 0.7576, 'eval_auc': 0.8934, 'eval_f1_invasao': 0.75, 'eval_f1_violencia': 0.7647, 'eval_runtime': 0.3046, 'eval_samples_per_second': 108.337, 'eval_steps_per_second': 16.415, 'epoch': 30.0}


# **6. RESULTADOS FINAIS DA VALIDAÇÃO CRUZADA**

In [29]:
print("\n\n==================== RESULTADOS FINAIS K-FOLD CROSS-VALIDATION ====================")

if all_fold_test_metrics:
    # Coletar todas as métricas de todos os folds
    df_metrics = pd.DataFrame(all_fold_test_metrics)

    # Calcular médias e desvios padrão
    avg_metrics = df_metrics.mean(numeric_only=True).round(4).to_dict()
    std_metrics = df_metrics.std(numeric_only=True).round(4).to_dict()

    print("Métricas Médias (e Desvio Padrão) em todos os Folds:")
    for metric, avg_value in avg_metrics.items():
        # Excluir métricas de tempo/desempenho por segundo da apresentação principal se desejar
        if 'runtime' not in metric and 'samples_per_second' not in metric and 'steps_per_second' not in metric:
            print(f"{metric}: {avg_value} (± {std_metrics.get(metric, 0)})")
else:
    print("Nenhum resultado de fold foi coletado. Ocorreram erros em todos os folds.")

print("\n✅ K-Fold Cross-Validation Concluído.")



==================== RESULTADOS FINAIS K-FOLD CROSS-VALIDATION ====================
Métricas Médias (e Desvio Padrão) em todos os Folds:
eval_loss: 0.4136 (± 0.0528)
eval_accuracy: 0.8221 (± 0.0764)
eval_auc: 0.9012 (± 0.0365)
eval_f1_invasao: 0.8128 (± 0.0786)
eval_f1_violencia: 0.8304 (± 0.0745)
epoch: 29.0 (± 2.2361)

✅ K-Fold Cross-Validation Concluído.


**7. SALVAR MODELO (Considerações Pós K-Fold)**

In [30]:
print("\nConsiderações sobre o salvamento do modelo pós K-Fold:")
print("A Validação Cruzada fornece uma estimativa robusta da performance.")
print("Para ter um modelo final para deployment, você pode:")
print("1. Re-treinar o modelo uma última vez com o dataset COMPLETO (todos os 150 registros).")
print("2. Selecionar o modelo de um dos folds que obteve o melhor desempenho na sua respectiva validação.")

# Exemplo de como re-treinar no dataset completo (se necessário para deployment)
# model_final = AutoModelForSequenceClassification.from_pretrained(
#     model_path, num_labels=2, id2label=id2label, label2id=label2id
# )
# for param in model_final.bert.parameters():
#     param.requires_grad = False
# for param in model_final.classifier.parameters():
#     param.requires_grad = True

# full_dataset_encodings = tokenize_function({"texto": df['texto'].tolist()})
# full_dataset_labels = df['classe'].map(label2id).tolist()
# full_custom_dataset = CustomDataset(pd.DataFrame(full_dataset_encodings), pd.Series(full_dataset_labels))

# trainer_final = Trainer(
#     model=model_final,
#     args=TrainingArguments(
#         output_dir="./bertimbau-final-model",
#         num_train_epochs=training_args.num_train_epochs, # Pode ser ajustado após ver os resultados da CV
#         learning_rate=training_args.learning_rate,
#         per_device_train_batch_size=training_args.per_device_train_batch_size,
#         save_strategy="epoch",
#         load_best_model_at_end=True,
#         metric_for_best_model="loss", # Ou outra métrica se houver eval_dataset
#         greater_is_better=False,
#         logging_steps=training_args.logging_steps,
#         fp16=training_args.fp16,
#         seed=training_args.seed,
#         report_to="none"
#     ),
#     train_dataset=full_custom_dataset,
#     data_collator=data_collator
# )
# print("\nIniciando re-treinamento no dataset completo para modelo final...")
# trainer_final.train()
# trainer_final.save_model("/content/drive/MyDrive/Pesquisa 2025/modelo_bertimbau_final_cv_retrained")
# tokenizer.save_pretrained("/content/drive/MyDrive/Pesquisa 2025/modelo_bertimbau_final_cv_retrained")
# print("Modelo final salvo no Google Drive após re-treinamento no dataset completo!")


Considerações sobre o salvamento do modelo pós K-Fold:
A Validação Cruzada fornece uma estimativa robusta da performance.
Para ter um modelo final para deployment, você pode:
1. Re-treinar o modelo uma última vez com o dataset COMPLETO (todos os 150 registros).
2. Selecionar o modelo de um dos folds que obteve o melhor desempenho na sua respectiva validação.
